Скачать датасет с фразами симпосонов - https://github.com/sujanjoejacob/Text-mining-with-Simpsons-Data
Обучить word2vec на фразах персонажей
Построить word cloud по самым частотным словам (top 1000)
Найти самые близкие слова для:
homer - marge + bart
bart - lisa + school
marge - homer + home
Попробовать построить классификатор сообщение Bart/Lisa c использованием полученых эмбеддингов

In [42]:
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
import numpy as np

In [ ]:
#file reading

In [5]:
data = pd.read_csv('simpsons_script_lines.csv', usecols=['normalized_text', 'raw_character_text'])

In [37]:
#functions
def stop_filter(res, stop_words):
    return [word for word in res if not word in stop_words] 

def lemmatization(res, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in res]

def bigram_creation(res):
    return [' '.join(x) for x in list(nltk.bigrams(res))]

In [30]:
nltk.download('stopwords')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/natalka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/natalka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/natalka/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#tokenization, removing stop words

In [25]:
stop_words = set(stopwords.words('english')) 
data['words'] = [word_tokenize(str(x)) for x in data['normalized_text']]
data['words'] = data['words'].apply(lambda x: stop_filter(x, stop_words))

In [ ]:
#lemmatization (also we can use stemming in this case)

In [31]:
data['words'] = data['words'].apply(lambda x: lemmatization(x, lemmatizer))

In [ ]:
#creating bigrams

In [38]:
data['words_bigram'] = data['words'].apply(lambda x: bigram_creation(x))

In [40]:
data['words'] = data['words'] + data['words_bigram']

In [43]:
#word2vec gensim

In [ ]:
sentences = np.concatenate(data['words'])